* **Linear decision boundaries:** The dataset comes from a Multivariate (K) Gaussian distribution with uncorrelated components and different means.

* Dataset for each class comes from a mixture of low-variance Gaussian distributions, with individual means themselves distributed as Gaussians.


LR gives a linear decision boundary (surprise!) -> **potentially high bias and low variance**.

This model works best for datasets generated from Multivariate Gaussian distributions with uncorrelated components and different means.

# Simple Linear Regression

* When there i a single input variable the method is referered to as simple linear regression.
$$ y = b_0 + b_1 x $$

We can estimate the coefficients as follows:
$$ 
b_1 = \frac{\sum_{i=1}^{n} (x_i - \mu_x )(y_i - \mu_y)}{\sum^{n}_{i=1} (x_i - \mu_x )^2} 
= \frac{covariance(x, y)}{variance(x)}
$$
$$
b_0 = \mu_y - b_1 \mu_x 
= mean(y) - b_1 * mean(x)
$$


* Mean       : $$ \mu_x = \frac{1}{n} \sum^{n}_{i=1} x_i $$
* Variance   : $$ \sum_{i=1}^{n} (x_i - \mu_x)^2 $$
* Covariance : Generalization of correlation to describe the relation between two or more values.
$$ \sum^{n}_{i=1} (x_i - \mu_x)(y_i - \mu_y) $$

## Quantities

In [1]:
def mean(values):
    return sum(values) / float(len(values))

def variance(values):
    mean_val = mean(values)
    variance = [ (val - mean_val)**2.0 for val in values ]
    return sum(variance)

def covariance(x, y):
    mean_x = mean(x)
    mean_y = mean(y)
    covar = 0.0
    for i in range(len(x)):
        covar += (x[i]-mean_x) * (y[i]-mean_y)
    return covar

In [2]:
dataset = [[1, 1], [2, 3], [4, 3], [3, 2], [5, 5]]                              
x = [row[0] for row in dataset]                                                 
y = [row[1] for row in dataset]                                                 
mean_x, mean_y = mean(x), mean(y)                                               
var_x, var_y = variance(x), variance(y)                         
print('x stats: mean=%.3f variance=%.3f' % (mean_x, var_x))                     
print('y stats: mean=%.3f variance=%.3f' % (mean_y, var_y))

covar = covariance(x, y)
print( 'Covariance: %.3f ' % (covar))

x stats: mean=3.000 variance=10.000
y stats: mean=2.800 variance=8.800
Covariance: 8.000 


## Coefficients

In [3]:
def coefficients(dataset):
    x = [row[0] for row in dataset]
    y = [row[1] for row in dataset]
    x_mean, y_mean = mean(x), mean(y)
    b1 = covariance(x, y) / variance(x)
    b0 = y_mean - b1*x_mean
    return b0, b1

In [4]:
b0, b1 = coefficients(dataset)
print( ' Coefficients: B0={:.3f}, B1={:.3f}'.format(b0, b1))

 Coefficients: B0=0.400, B1=0.800


## Putting it all together

In [5]:
def simple_linear_regression(train, test):
    predictions = []
    b0, b1 = coefficients(train)
    for row in test:
        yhat = b0 + b1*row[0]
        predictions.append( yhat )
    return predictions

In [6]:
from Getting_Started import rmse_metric

def evaluate_algorithm(dataset, algorithm):
    test_set = []
    for row in dataset:
        row_copy = list(row)
        row_copy[-1] = None
        test_set.append(row_copy)
    predicted = algorithm(dataset, test_set)
    print(predicted)
    actual = [row[-1] for row in dataset]
    rmse = rmse_metric(actual, predicted)
    return rmse

In [7]:
rmse = evaluate_algorithm(dataset, simple_linear_regression)
print( ' RMSE: %.3f ' % (rmse))

[1.1999999999999995, 1.9999999999999996, 3.5999999999999996, 2.8, 4.3999999999999995]
 RMSE: 0.693 


# Multivariate Linear Regression

Given a vector of inputs $X^T = (X_1, ..., X_p)$, $X$ being a column vector, one can predict the output of $Y$ by using the following model:
$$
\hat{Y} = \hat{\beta_0} + \sum^{p}_{j=1} X_j \hat{\beta_j}
$$
$p$ is the number of features.
$\hat{\beta_0}$ is the intercept or **bias** term.

We can include the constant variable 1 in $X$ in order to include $\hat{\beta_0}$ with the vector of coefficients $\hat{\beta}$ so that our linear model can be writen as an inner product,
$$
\hat{Y} = X^T \hat{\beta}
$$

We can interpret our model as a function over a $p$-dimensional input space, $f(X) = X^T \hat{\beta}$ with its gradient $f^{\prime} (X) = \hat{\beta} $.
$\beta$ (we are dropping the hat) is thus a vector in input space that points in the steepest uphill direction.

To fit the linear model to a dataset we will use the method of least squares.
We pick the coefficients $\beta$ that **minimize the residual sum of squares**
$$
RSS(\beta ) =\sum^{N}_{i=1} (y_i - x_{i}^{T}\beta )^2
$$

In matrix notation,
$$
RSS(\beta ) = (\mathbf{y - X}\beta)^T (\mathbf{y - X}\beta),
$$
$\mathbf{X}$ is an $N \times p$ matrix where each row corresponds to an input vector, and $\mathbf{y}$ is a $N$-vector of outputs for the training set.
Because the RSS is a quadratic function its minimum always exists, although it may not be unique.

In order to continue, let's differentiate $RSS(\beta )$ with respect to $\beta$. This will give us the set of equations,
$$
\mathbf{X}^T (\mathbf{y-X}\beta) = 0
$$

In the case that $\mathbf{X}^T \mathbf{X}$ is nonsingular, we then obtain 
$$
\hat{\beta} = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}
$$

## Stochastic Gradient Descent for Linear Regression

In [11]:
def predict(row, coefficients):
    yhat = coefficients[0]
    for i in range(len(row)-1):
        yhat += coefficients[i+1] * row[i]
    return yhat

def coefficients_sgd(train, l_rate, n_epoch):
    coef = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        sum_error = 0.0
        for row in train:
            yhat = predict(row, coef)
            error = yhat - row[-1]
            sum_error += error**2.0
            # updates 
            coef[0] = coef[0] - l_rate * error
            for i in range(len(row)-1):
                coef[i+1] = coef[i+1] - l_rate * error * row[i]
    return coef

def linear_regression_sgd(train, test, l_rate, n_epoch):                        
    predictions = []                                                    
    coef = coefficients_sgd(train, l_rate, n_epoch)                         
    for row in test:                                                        
        yhat = predict(row, coef)                                       
        predictions.append(yhat)                                        
    return(predictions)

In [12]:
def evaluate_algorithm(dataset, algorithm, n_folds, *args):                     
    folds = cross_validation_split(dataset, n_folds)                        
    scores = []                                                         
    for fold in folds:                                                      
        train_set = list(folds)                                         
        train_set.remove(fold)                                          
        train_set = sum(train_set, [])                                  
        test_set = list()                                               
        for row in fold:                                                
            row_copy = list(row)                                    
            test_set.append(row_copy)                               
            row_copy[-1] = None                                     
        predicted = algorithm(train_set, test_set, *args)               
        actual = [row[-1] for row in fold]                              
        rmse = rmse_metric(actual, predicted)                           
        scores.append(rmse)                                             
    return scores

In [13]:
from Getting_Started import load_csv, str_column_to_float, normalize_dataset, cross_validation_split

filename = '../datasets/winequality-white.csv'                                              
dataset = load_csv(filename)                                                    
for i in range(len(dataset[0])):                                                
    str_column_to_float(dataset, i)                                         
# normalize                                                                                                                     
dataset = normalize_dataset(dataset)

# evaluate algorithm                                                            
n_folds = 5                                                                     
l_rate = 0.01                                                                   
n_epoch = 50                                                                    
scores = evaluate_algorithm(dataset, linear_regression_sgd, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)                                                    
print('Mean RMSE: %.3f' % (sum(scores)/float(len(scores)))) 

Scores: [0.12799871928083031, 0.12495030020491166, 0.12225645975212905, 0.12914294667502924, 0.12603189216806354]
Mean RMSE: 0.126
